In [4]:
# import required packages
import cv2
import numpy as np
import os
'pip install ultralytics'
from ultralytics import YOLO
import math 
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath("../../")
sys.path.append(parent_dir)
from Update_Git import git_add, git_commit, git_push

### GitHub

In [ ]:
path = os.path.join(current_dir, "yolo_1.ipynb")
for i in range(27):
    if os.path.exists(path):
        git_add(path)
        git_commit("Add yolo_1.ipynb")
        git_push("main")
    else:
        print("File does not exist.")

Error executing command: ['git', 'commit', '-m', 'Add yolo_1.ipynb']
Error message: 
Error executing command: ['git', 'commit', '-m', 'Add yolo_1.ipynb']
Error message: 
Error executing command: ['git', 'commit', '-m', 'Add yolo_1.ipynb']
Error message: 
Error executing command: ['git', 'commit', '-m', 'Add yolo_1.ipynb']
Error message: 
Error executing command: ['git', 'commit', '-m', 'Add yolo_1.ipynb']
Error message: 
Error executing command: ['git', 'commit', '-m', 'Add yolo_1.ipynb']
Error message: 
Error executing command: ['git', 'commit', '-m', 'Add yolo_1.ipynb']
Error message: 
Error executing command: ['git', 'commit', '-m', 'Add yolo_1.ipynb']
Error message: 
Error executing command: ['git', 'commit', '-m', 'Add yolo_1.ipynb']
Error message: 
Error executing command: ['git', 'commit', '-m', 'Add yolo_1.ipynb']
Error message: 


### Preparing the input

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    ret, img= cap.read()
    cv2.imshow('Webcam', img)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### Model

In [21]:
model = YOLO("yolo-Weights/yolov8n.pt")

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

In [22]:
while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 144.0ms
Confidence ---> 0.91
Class name --> person
Speed: 4.0ms preprocess, 144.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 151.0ms
Confidence ---> 0.9
Class name --> person
Speed: 3.0ms preprocess, 151.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 102.5ms
Confidence ---> 0.91
Class name --> person
Speed: 1.0ms preprocess, 102.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 114.5ms
Confidence ---> 0.91
Class name --> person
Speed: 2.0ms preprocess, 114.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 108.5ms
Confidence ---> 0.92
Class name --> person
Speed: 1.0ms preprocess, 108.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 105.5ms
Confidence ---> 0.91
Class name --> person
Speed: 2.0ms preprocess, 105.5ms inference, 1.0ms postprocess per image a